In [2]:
import numpy as np
import math

In [3]:
#given long sequence of base pairs, and k for the length of the k-mer
#output a dictionary that will keep track of the counts of each k-mer seen, using their numeric value
def sequenceToNumber(sequence, k):
	basePairs = {'A' : 0, 'a' : 0, 'C': 1, 'c': 1, 'G': 2, 'g': 2, 'T': 3, 't': 3}
	# referenceArray = [0] * (4**k)
	referenceArray = {}

	# print(sequence)
	kmerToNumber = 0
	kmer = sequence[0: k]
	firstBasePair = 0

	#generate kmertToNumber for first kmer base pairs 
	for idx, letter in enumerate(kmer):
		kmerToNumber += basePairs[letter] * (4 ** (k - idx - 1))



	if kmerToNumber in referenceArray:
		referenceArray[kmerToNumber] += 1
	else:
		referenceArray[kmerToNumber] = 1


	#generate kmerToNumber without going through all k base pairs
	for i in range(k, len(sequence)):

		firstBasePair = basePairs[sequence[i - k]] * 4 ** (k - 1)
		# print("firstBasePair: ", firstBasePair)
		#subtract the first base pair's value
		kmerToNumber -= firstBasePair
		kmerToNumber *= 4
		kmerToNumber += basePairs[sequence[i]]

		#save the value of the new first base pair in new k-mer to subtract later
		

		# print("kmerToNumber: ", kmerToNumber)
		if kmerToNumber in referenceArray:
			referenceArray[kmerToNumber] += 1
		else:
			referenceArray[kmerToNumber] = 1

	return referenceArray

In [4]:
validBasePairs = ['A', 'a', 'C', 'c', 'G', 'g', 'T', 't']

In [7]:
filepath = 'hpv68'
viralSequence = None
with open(filepath) as fp:
	for line in fp:
		line = line.strip()
		viralSequence = sequenceToNumber(line, 15)
print(viralSequence)

{244961233: 1, 979844932: 1, 698154256: 1, 645133377: 1, 433049861: 1, 658457621: 1, 486346839: 1, 871645534: 1, 265356664: 1, 1061426658: 1, 1024481162: 1, 876699176: 1, 285571232: 1, 68543105: 1, 274172422: 1, 22947866: 1, 91791465: 1, 367165861: 1, 394921620: 1, 505944659: 1, 950036812: 1, 578921777: 1, 168203460: 1, 672813840: 1, 543771712: 1, 27603203: 1, 110412815: 1, 441651262: 1, 692863225: 1, 623969253: 1, 348393364: 1, 319831634: 1, 205584712: 1, 822338849: 1, 68129925: 1, 272519703: 1, 16336990: 1, 65347963: 1, 261391854: 1, 1045567417: 1, 961044196: 1, 622951314: 1, 344321610: 1, 303544616: 1, 140436641: 1, 561746564: 1, 99502611: 1, 398010447: 1, 518299966: 1, 999458042: 1, 776606696: 1, 958943137: 1, 614547076: 1, 310704657: 1, 169076807: 1, 676307228: 1, 557745265: 1, 83497412: 1, 333989651: 1, 262216783: 1, 1048867134: 1, 974243065: 1, 675746788: 1, 555503507: 1, 74530382: 1, 298121528: 1, 118744289: 1, 474977158: 1, 826166811: 1, 83441775: 1, 333767100: 1, 261326577: 1

In [ ]:
genomeFilepath = "../projectSequences/HumanGenome/chr11.fa"
humanStringSequence = ""
humanSequence = None
with open(genomeFilepath) as fp:
	for line in fp:
		line = line.strip()
		for char in line:
			if char in validBasePairs:
				humanStringSequence += char.upper()
	humanSequence = sequenceToNumber(humanStringSequence, 15)

print(humanSequence)